In [189]:
import pandas as pd
import numpy as np

In [190]:
path = "C:\\Users\\97250\\Desktop\\תואר הנדסה\\שנה ג' סמסטר ב\\data mining\\Tasks\\2\\matala2_cosmetics_2019-Nov.csv"
data = pd.read_csv(path,low_memory=False)
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:02,view,5802432,1487580009286590000,NaN,NaN,0.32,562076640,09fafd6c-6c99-46b1-834f-33527f4de241
1,2019-11-01 00:00:09,cart,5844397,1487580006317030000,NaN,NaN,2.38 sale,553329724,2067216c-31b5-455d-a1cc-af0575a34ffb
2,2019-11-01 00:00:10,view,5837166,1783999064103190000,NaN,pnb,22.22,556138645,57ed222e-a54a-4907-9944-5a875c2d7f4f
3,2019-11-01 00:00:11,cart,5876812,1487580010100290000,NaN,jessnail,3.16,564506666,186c1951-8052-4b37-adce-dd9644b1d5f7
4,2019-11-01 00:00:24,remove_from_cart,5826182,1487580007483040000,NaN,NaN,3.33,553329724,2067216c-31b5-455d-a1cc-af0575a34ffb


## 1. duration_to_next_event 

to manipulate our event_time we need that the dtype of that column will be **datetime**, lets check the dtype

In [191]:
data.dtypes

event_time       object
event_type       object
product_id        int64
category_id       int64
category_code    object
brand            object
price            object
user_id           int64
user_session     object
dtype: object

We can see that event_time standing on **object** dtype, lets convert it **&** sort the data by user_session.

In [192]:
data['event_time']= pd.to_datetime(data['event_time'])
data.dtypes

event_time       datetime64[ns]
event_type               object
product_id                int64
category_id               int64
category_code            object
brand                    object
price                    object
user_id                   int64
user_session             object
dtype: object

In [193]:
data = data.sort_values(by=['user_session','event_time'],ascending = True)

Now, lets use diff() to calculate time between events (We can see that the data already sorted so no action needed)

In [194]:
data['duration_to_next_event'] = (data['event_time'].shift(-1)-data['event_time']).dt.seconds
data.loc[data['user_session'].shift(-1) != data['user_session'], 'duration_to_next_event'] = None
data['duration_to_next_event'] = data['duration_to_next_event'].fillna(0)
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,duration_to_next_event
522763,2019-11-05 07:57:05,view,5829298,1487580011585070000,NaN,farmstay,0.79,556321594,00000dd2-0f5d-4fc9-9d6b-2fc8c7514b04,0.0
284392,2019-11-03 12:45:44,view,5846437,1487580013950660000,NaN,browxenna,45.24,473600495,00005ac0-5c22-4756-bb12-31b90af75fa3,11.0
284416,2019-11-03 12:45:55,view,5846436,1487580013950660000,NaN,browxenna,45.24,473600495,00005ac0-5c22-4756-bb12-31b90af75fa3,0.0
144335,2019-11-02 09:13:51,view,5887753,1487580013279570000,NaN,NaN,3.56,557616099,0000d358-49a1-4a54-b402-08205c35981c,0.0
853411,2019-11-07 11:18:15,view,59996,1487580012927250000,NaN,NaN,4.13,463202709,0000d72e-be5f-4a03-9f17-6ac81584f177,71.0


## 2. funnel_number

We can see our data sorted according to event_time column, In order to calculate the sessions we will need to make a Temporary data frame  
that sorted by user_id, In addition we will add the sessions value to a dict and than we will map them by user_id.  
**In addition, We will need to change event_time and drop the hours & seconds to calculate days difference for the fuunel column**

In [195]:
data_sorted = data.sort_values(by=['user_id','event_time'])
data_sorted['event_date'] = data_sorted['event_time'].dt.date

Now, lets create days_between column, we will use it to calculate the cumulative days for the sessions.  
**important! we have to make a condition between different users to ensure we will not mistakely calulate the days between them**

In [196]:
data_sorted['days_between'] = (data_sorted.groupby('user_id')['event_date']
                                        .diff().apply(pd.Timedelta).dt.days
                                        .fillna(0))
data_sorted['days_between'].fillna(0, inplace=True)
data_sorted.loc[data_sorted['user_id'] != data_sorted['user_id'].shift(), 'days_between'] = None

finally, we will generate formula that including inside cumsum(), cumsum function calculate the cumulative values.  
Then adjust it to the original df

In [ ]:
data_sorted['funnel_number'] = (data_sorted.groupby('user_id')['days_between']
                                       .transform(lambda x: (x.cumsum() // 5) + 1))
data_sorted['funnel_number'] = data_sorted['funnel_number'].fillna(1)

In [ ]:
data_sorted.loc[data['user_id'] == 28048440].head(15)

## 3. index_in_funnel 

In [ ]:
data_sorted['index_in_funnel'] = data_sorted.groupby(['user_id', 'funnel_number']).cumcount() + 1

Lets take some userID for example to see if it works

In [ ]:
data_sorted.loc[data['user_id'] == 28048440  ].head(10)

# 4. re

In [ ]:
import re


In [ ]:
data_sorted['price'] = data_sorted.price.apply(lambda x: "".join(re.findall('[.,0-9]', x))).astype(float)
data_sorted['price']

# 5. Events

In [ ]:
data_sorted.event_type.hist()

# 6. session_data 

In [ ]:
grouped = data_sorted.groupby(['user_id', 'user_session', 'funnel_number'])
session_data = pd.DataFrame()

In [ ]:
session_data['user_id'] = grouped['user_id'].first()
session_data['user_session'] = grouped['user_session'].first()
session_data['funnel_number'] = grouped['funnel_number'].first()
session_data['index_in_funnel'] = grouped['index_in_funnel'].apply(list)

**column e**

In [ ]:
session_data['total_events'] = data_sorted.groupby(['user_id', 'user_session'])['user_session'].count()

**column f**

In [ ]:
session_data['seconds_between'] = data_sorted.groupby(['user_id','user_session'])['duration_to_next_event'].sum()

**column g**

In [ ]:
view_sort = data_sorted[data_sorted['event_type'] == 'view']
session_data['list_of_viewed'] = view_sort.groupby(['user_id', 'user_session'])['product_id'].apply(list)

**column h**

In [ ]:
cart_sort = data_sorted[data_sorted['event_type'] == 'cart']
session_data['list_of_added_to_cart'] = cart_sort.groupby(['user_id', 'user_session'])['product_id'].apply(list)

**column i**

In [ ]:
purchase_sort = data_sorted[data_sorted['event_type'] == 'purchase']
session_data['list_of_purchased '] = cart_sort.groupby(['user_id', 'user_session'])['product_id'].apply(list)

In [ ]:
session_data.head(8)